Grabbing list of artists

In [ ]:
## Important imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from dotenv import load_dotenv
import os
import pandas as pd
# Set up authentication with your Spotify Developer credentials
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
))


In [ ]:
import pandas as pd

df = pd.read_csv("listeners.csv")
df = df.drop(columns=['Listeners', 'Daily Trend', 'Peak', 'PkListeners'])
df.to_csv("artists.csv", index=False)
df.head()

Scrapping artists URI's

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from dotenv import load_dotenv
import os
import pandas as pd

# Set up authentication with your Spotify Developer credentials
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id= client_id,
    client_secret= client_secret
))

df = pd.read_csv("artists.csv")
artist_names = df['Artist'].tolist()

artists_list = []
uris_list = []

for artist_name in artist_names:
    results = sp.search(q='artist:' + artist_name, type='artist', limit=1)
    items = results['artists']['items']

    if items:
        artist = items[0]
        artist_uri = artist['uri']
        print(f"Artist URI for '{artist_name}': {artist_uri}")
        # Append to lists
        artists_list.append(artist_name)
        uris_list.append(artist_uri)
    else:
        print(f"Artist '{artist_name}' not found.")
        # Append name with None URI
        artists_list.append(artist_name)
        uris_list.append(None)

# Create a DataFrame with the results
result_df = pd.DataFrame({
    "Artists": artists_list,
    "URI": uris_list
})

# Save to CSV
result_df.to_csv("artist_uris.csv", index=False)

Getting Track ID's

In [ ]:
# Load CSV with columns 'Artist' and 'URI'
df = pd.read_csv('artists_uris_cleaned.csv')

# Prepare lists to collect results
track_names = []
album_names = []
artist_names_all = []
track_uris = []

for artist_name, artist_uri in zip(df['Artists'], df['URI']):
    artist_id = artist_uri.split(":")[-1]
    
    # Get all albums by the artist (limit and offset used to get all pages)
    albums = []
    offset = 0
    while True:
        results = sp.artist_albums(artist_id, album_type='album,single', limit=50, offset=offset)
        albums.extend(results['items'])
        if results['next'] is None:
            break
        offset += 50

    # For each album get the tracks
    for album in albums:
        album_name = album['name']
        album_id = album['id']

        # Get tracks for the album
        tracks = []
        track_offset = 0
        while True:
            track_results = sp.album_tracks(album_id, limit=50, offset=track_offset)
            tracks.extend(track_results['items'])
            if track_results['next'] is None:
                break
            track_offset += 50

        # For each track, get the track name, artists, and URI
        for track in tracks:
            track_name = track['name']
            track_uri = track['uri']
            artists = [artist['name'] for artist in track['artists']]
            artists_str = ", ".join(artists)

            track_names.append(track_name)
            track_uris.append(track_uri)
            album_names.append(album_name)
            artist_names_all.append(artists_str)

            # Print the track info as it is added
            print(f"Added Track: '{track_name}' by {artists_str} from Album: '{album_name}', URI: {track_uri}")

    print(f"Fetched {len(track_names)} tracks so far for artist '{artist_name}'")

# Create a DataFrame with the collected data
result_df = pd.DataFrame({
    'Track Name': track_names,
    'Album Name': album_names,
    'Artist(s)': artist_names_all,
    'Track URI': track_uris
})

# Save to CSV
result_df.to_csv('artist_tracks_album_info.csv', index=False)

print(f"Saved {len(result_df)} tracks to artist_tracks_album_info.csv")


In [ ]:
## Trying to only take one album from each artist
# Load CSV with columns 'Artist' and 'URI'
df = pd.read_csv('artists_uris_cleaned.csv')

# Prepare lists to collect results
track_names = []
album_names = []
artist_names_all = []

for artist_name, artist_uri in zip(df['Artists'], df['URI']):
    artist_id = artist_uri.split(":")[-1]
    
    # Get the first album only (no pagination)
    results = sp.artist_albums(artist_id, album_type='album,single', limit=1, offset=0)
    albums = results['items']

    if not albums:
        print(f"No albums found for artist '{artist_name}'")
        continue

    album = albums[0]
    album_name = album['name']
    album_id = album['id']

    # Get tracks for the album
    tracks = []
    track_offset = 0
    while True:
        track_results = sp.album_tracks(album_id, limit=50, offset=track_offset)
        tracks.extend(track_results['items'])
        if track_results['next'] is None:
            break
        track_offset += 50

    # For each track, get the track name and artists only (remove URI)
    for track in tracks:
        track_name = track['name']
        artists = [artist['name'] for artist in track['artists']]
        artists_str = ", ".join(artists)

        track_names.append(track_name)
        album_names.append(album_name)
        artist_names_all.append(artists_str)

        # Print the track info as it is added (no URI included)
        print(f"Added Track: '{track_name}' by {artists_str} from Album: '{album_name}'")

    print(f"Fetched {len(track_names)} tracks so far for artist '{artist_name}'")

# Create a DataFrame with the collected data (no URI column)
result_df = pd.DataFrame({
    'Track Name': track_names,
    'Album Name': album_names,
    'Artist(s)': artist_names_all
})

# Save to CSV
result_df.to_csv('artist_tracks_album_info.csv', index=False)

print(f"Saved {len(result_df)} tracks to artist_tracks_album_info.csv")


In [ ]:
from spotipy.oauth2 import SpotifyOAuth
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id= client_id,
                                               client_secret= client_secret,
                                               redirect_uri="https://google.com/",
                                               scope="user-library-read"))

Tring MusicBrainz API, Spotify API is deprecated

In [ ]:
import requests

def get_recording_mbid(artist_name, track_name):
    """Search MusicBrainz for recording MBID by artist and track name."""
    query = f'artist:"{artist_name}" AND recording:"{track_name}"'
    url = "https://musicbrainz.org/ws/2/recording/"
    headers = {
        'User-Agent': 'ai-dj/1.0 (donutzay@gmail.com)'
    }
    params = {
        'query': query,
        'fmt': 'json',
        'limit': 1
    }
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
    if data['recordings']:
        mbid = data['recordings'][0]['id']
        print(f"Found recording MBID: {mbid}")
        return mbid
    else:
        print("Recording MBID not found")
        return None

def get_acoustic_features(mbid):
    """Fetch audio features from AcousticBrainz using MusicBrainz recording MBID."""
    url = f"https://acousticbrainz.org/api/v1/{mbid}/low-level"
    response = requests.get(url)
    if response.status_code == 404:
        print(f"No AcousticBrainz data found for MBID {mbid}")
        return None
    response.raise_for_status()
    features = response.json()
    print(f"Retrieved AcousticBrainz features for MBID: {mbid}")
    return features

# Example usage in Jupyter notebook:
artist = "Drake"
track = "Controlla"

mbid = get_recording_mbid(artist, track)
if mbid:
    features = get_acoustic_features(mbid)
    if features:
        # Display some keys from the features for inspection
        print("Some AcousticBrainz feature keys:")
        print(list(features.keys())[:10])
else:
    print("Could not retrieve AcousticBrainz features.")


Uses libRosa and Deezer to download a preview of a song and perform feature extraction

In [ ]:
import requests
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

def get_deezer_preview_url(track_name):
    url = f"https://api.deezer.com/search/track?q={track_name}"
    response = requests.get(url)
    data = response.json()
    if data['data']:
        first_track = data['data'][0]
        preview_url = first_track.get('preview')
        return preview_url if preview_url else None
    return None

def download_preview(preview_url, output_filename):
    response = requests.get(preview_url)
    with open(output_filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded preview to {output_filename}")

def plot_mel_spectrogram(filename):
    y, sr = librosa.load(filename, sr=None)
    print(f"Audio loaded with {len(y)} samples at {sr} Hz")
    mel_spect = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    mel_spect_db = librosa.power_to_db(mel_spect, ref=np.max)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spect_db, sr=sr, x_axis='time', y_axis='mel', fmax=8000)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram from MP3')
    plt.tight_layout()
    plt.show()

def feature_extract(track_name, tmp_filename='preview.mp3'):
    preview_url = get_deezer_preview_url(track_name)
    if preview_url:
        print("Preview URL:", preview_url)
        download_preview(preview_url, tmp_filename)
        plot_mel_spectrogram(tmp_filename)
        os.remove(tmp_filename)
        print(f"Deleted temporary file {tmp_filename}")
    else:
        print("Preview URL not available or track not found.")

# Example usage
feature_extract("Headlines")


Using deezer to get songs to use

In [42]:


def get_artist_id(artist_name):
    url = f"https://api.deezer.com/search/artist?q={artist_name}"
    data = requests.get(url).json()
    if data['data']:
        return data['data'][0]['id']
    return None

def get_albums(artist_id):
    url = f"https://api.deezer.com/artist/{artist_id}/albums"
    data = requests.get(url).json()
    return data['data'] if 'data' in data else []

def get_tracks(album_id):
    url = f"https://api.deezer.com/album/{album_id}/tracks"
    data = requests.get(url).json()
    return data['data'] if 'data' in data else []

def get_all_tracks_by_artist(artist_name):
    artist_id = get_artist_id(artist_name)
    if not artist_id:
        print("Artist not found")
        return []

    albums = get_albums(artist_id)
    all_tracks = []
    for album in albums:
        tracks = get_tracks(album['id'])
        for track in tracks:
            all_tracks.append({
                'track_title': track['title'],
                'preview_url': track.get('preview'),
                'album_title': album['title']
            })
    return all_tracks


df = pd.read_csv("artists_uris_cleaned.csv")
artist_names = df['Artists']

first_artist = artist_names.iloc[0]  

tracks = get_all_tracks_by_artist(first_artist)
df_tracks = pd.DataFrame(tracks)
df_tracks.to_csv('artist_tracks.csv', index=False)

